In [1]:
import numpy as np
from define import qcode as qc
from define import globalvars as gv
from define.randchans import RandomUnitary
from define.QECCLfid.utils import Dot, Dagger, Kron
from define.QECCLfid.multi_qubit_kraus import get_process_correlated

In [2]:
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)

In [3]:
# Pr(synd) = sum_s,s' GS_ss' P(synd)_s'

In [4]:
def LoadChannel():
    return np.loadtxt("./../input/debug_testing/physical.txt").reshape(256, 256)
def LoadProjector():
    return np.loadtxt("./../input/debug_testing/SS.txt").reshape(64, 64)

In [5]:
def SyndromeTest(GS, P):
    # Test if any syndrome probabilities are negative.
    norm = 0
    stop = 0
    for synd in range(64):
        Pr_s = 0
        if stop == 0:
            print("P[{}, :] has {} 1s and {} -1s.".format(synd, np.count_nonzero(P[synd, :] == 1), np.count_nonzero(P[synd, :] == -1)))
        Pr_s = np.sum(np.dot(GS, P[synd, :]))/64
        norm = norm + Pr_s
        if stop == 0:
            print("Pr({}) = {}".format(synd, Pr_s))
        if Pr_s < 0:
            stop = 1
            print("Negative Pr(s) encountered.")
    print("Sum of all syndrome probabilities: {}.".format(norm))
    return None

In [6]:
# We want to construct the S x S part of the process matrix for a random independent unitary channel.
# G_ij = prod_q Tr[ U_q  (P_i)_q U^dag_q  (P_j)_q ]

In [7]:
def GetRandUnitaryProcess(nq):
    U = np.zeros((nq, 2, 2), dtype=np.complex128)
    for q in range(nq):
        U[q, :, :] = RandomUnitary(prox=0.1, dim=2)
    S = qc.GenerateGroup(qcode.S)
    GS = np.zeros((S.shape[0], S.shape[0]), dtype = np.double)
    for i in range(S.shape[0]):
        for j in range(S.shape[0]):
            contrib = 1 + 0 * 1j
            for q in range(nq):
                contrib = contrib * np.trace(Dot(U[q, :, :], gv.Pauli[S[i, q], :, :], Dagger(U[q, :, :]), gv.Pauli[S[j, q], :, :]))
            GS[i, j] = np.real(contrib)/np.power(2, nq)
    return (U, GS)

In [8]:
GS = LoadChannel()[:64, :64]
P = LoadProjector()
SyndromeTest(GS, P)

P[0, :] has 64 1s and 0 -1s.
Pr(0) = 0.01776166873934213
P[1, :] has 32 1s and 32 -1s.
Pr(1) = 0.029819225617637606
P[2, :] has 32 1s and 32 -1s.
Pr(2) = 0.019050070006065685
P[3, :] has 32 1s and 32 -1s.
Pr(3) = 0.01428886061133896
P[4, :] has 32 1s and 32 -1s.
Pr(4) = -0.00011155629058838545
Negative Pr(s) encountered.
Sum of all syndrome probabilities: 0.9999999999999994.


In [9]:
(U, GS) = GetRandUnitaryProcess(7)
P = LoadProjector()
SyndromeTest(GS, P)

P[0, :] has 64 1s and 0 -1s.
Pr(0) = 3.338919252020289e-08
P[1, :] has 32 1s and 32 -1s.
Pr(1) = 4.3587117259415165e-07
P[2, :] has 32 1s and 32 -1s.
Pr(2) = 7.213913753509327e-06
P[3, :] has 32 1s and 32 -1s.
Pr(3) = 1.2181404236352722e-07
P[4, :] has 32 1s and 32 -1s.
Pr(4) = 3.6198340234768617e-06
P[5, :] has 32 1s and 32 -1s.
Pr(5) = 6.13625626500261e-07
P[6, :] has 32 1s and 32 -1s.
Pr(6) = 0.00046997860806691216
P[7, :] has 32 1s and 32 -1s.
Pr(7) = 1.8943119025650879e-06
P[8, :] has 32 1s and 32 -1s.
Pr(8) = 7.747475300190487e-10
P[9, :] has 32 1s and 32 -1s.
Pr(9) = 5.313791413483204e-08
P[10, :] has 32 1s and 32 -1s.
Pr(10) = 5.792251055528053e-09
P[11, :] has 32 1s and 32 -1s.
Pr(11) = 2.6154552155467636e-07
P[12, :] has 32 1s and 32 -1s.
Pr(12) = 2.215284689889574e-08
P[13, :] has 32 1s and 32 -1s.
Pr(13) = 1.3237191710639695e-07
P[14, :] has 32 1s and 32 -1s.
Pr(14) = 6.786137339565013e-08
P[15, :] has 32 1s and 32 -1s.
Pr(15) = 1.7060210468815284e-05
P[16, :] has 32 1s and

In [10]:
kraus_dict = {0:(tuple(range(qcode.N)), [Kron(*U)])}

In [11]:
G_frontend = get_process_correlated(qcode, kraus_dict)

In [12]:
GS_frontend = G_frontend.reshape(256, 256)[:64, :64]

In [13]:
np.allclose(GS_frontend, GS)

True